# Surgery to OMOP

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

This is mainly mapped to procedure occurrence and measurement following https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=449874117#gid=449874117

### Doubts

THE OTHER SURGERIES ARE MEASUREMENTNS NOW!

Should we also create a visit occurrence?

Since this is a "treatment," we could create an Episode (which would involve storing data at a higher level), but since it is allowed in OMOP and we are instructed to do so in the mapping, we will simply use Procedure Occurrence (a lower level)

NECK AND METASTASIS SURGERY NEED TO BE FURHTER CHECKED TO SEE IF OUR IDEA IS RIGHT: measurement with concept id= the one in the mapping, and value as concept id = https://athena.ohdsi.org/search-terms/terms/45884591 . For these, we will link them to the main surgery event (as we are doing with all the other measurements) using the meas_event_field_concept_id


In [ ]:
surgery_IDEA4RC = pd.read_csv("./IDEA4RC-data/surgeryIDEA4RC.csv")
surgery_IDEA4RC.head(5)

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Surgery to Procedure Ocurrence

Since this is a "treatment," we could create an Episode (which would involve storing data at a higher level), but since it is allowed in OMOP and we are instructed to do so in the mapping, we will simply use Procedure Occurrence (a lower level)

An important thing about this is that we should find a way to keep a reference from H&N and Metastasis surgeries to the "main" surgery: we will use the id to create the needed measurements for the other surgeries, and the meas_event_field_concept_id

In [ ]:
from uuid import uuid4


queryEVPO="""
    INSERT INTO omopcdm.episode_event (episode_id,event_id,episode_event_field_concept_id)
    VALUES (%s, %s, %s);
"""
df_tables=surgery_IDEA4RC
df_tables['newId']=None
query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM episode_event e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1;

"""
sqlProcedure="""
    INSERT INTO omopcdm.procedure_ocurrence (procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_type_concept_id, provider_id)
    VALUES (%s, %s, %s,%s,%s, %s);
"""
queryDate="""
    SELECT episode_start_date
    FROM cancer_episode
    WHERE id = %s;
"""
sqlGetEpisode = """
    SELECT c.id 
    FROM episode c
    WHERE c.person_id = %s
    AND c.episode_start_date = %s
    LIMIT 1;
"""
cols= {
    "SURGICAL_PROCEDURE": 4301351,
    "LOCAL_EXCISION": 4315400,
    "AMPUTATION": 4217482,
    "LIMB_OPERATION": 4181193,
    "PALLIATIVE": 4179711,
    "CURATIVE": 4162591,
    "EXTERNAL_OPEN": 4236080,
    "TRANSNASAL": 4050374,
    "TRANSORBITAL": 4044897,
    "TRANSORAL": 4038679,
    "CRANIOFACIAL": 4333339,
    "NO_RESIDUAL_TUMOR": 1634643,
    "MICROSCOPIC_RESIDUAL_TUMOR": 1633801,
    "MACROSCOPIC_RESIDUAL_TUMOR": 1634484,
    "tumor_rupture": 36768904, #Since we have a vocabulary for a boolean, I will use it as a modifier
    "surgical_specimen_mitotic_count": 4227243,
    "GRADE_1": 1634371,
    "GRADE_2": 1634752,
    "GRADE_3": 1633749,
    "GRADE_H": 1635587,
    "GRADE_L": 1634085,
    "reconstruction": 3010988,
    "IPSILATERAL": 4112106,
    "BILATERAL_SAMPLING": 4117496,
    "CONTROLATERAL": 4112107,
    "site_of_surgery_on_metastasis_soft_tissue": 35225724,
    "site_of_surgery_on_metastasis_distant_lymph_nodes": 36769243,
    "site_of_surgery_on_metastasis_lung": 36770283,
    "site_of_surgery_on_metastasis_bone": 36769301,
    "site_of_surgery_on_metastasis_liver": 36770544,
    "site_of_surgery_on_metastasis_pleura": 35226258,             
    "site_of_surgery_on_metastasis_peritoneum": 35226253,
    "site_of_surgery_on_metastasis_brain": 36768862,
    "site_of_surgery_on_metastasis_other_viscera":  36769180,
    "site_of_surgery_on_metastasis_unknown": 4129922,
    "NO_COMPLICATION": 4132135,
    "CLA_DINDO_SCALE_I": 715974,
    "CLA_DINDO_SCALE_II": 715975,
    "CLA_DINDO_SCALE_III": 715976,
    "CLA_DINDO_SCALE_IIIA": 715977,
    "CLA_DINDO_SCALE_IIIB": 715978,
    "CLA_DINDO_SCALE_IV": 715979,
    "CLA_DINDO_SCALE_IVA": 715980,
    "CLA_DINDO_SCALE_IVB": 715981,
    "CLA_DINDO_SCALE_V": 715982
}
def toTableEpisodeEventProcedureOcurrence(idEpisode , idProcedureOcurrence):
    cur.execute(queryEVPO,(idEpisode,idProcedureOcurrence,1147082)) #1147082 or 1147810, not clear
    conn.commit()

for idx, row in df_tables.iterrows():
    curIDEA.execute(query, (row['Episode Event Reference']))
    res=curIDEA.fetchone()
    person_id=res
    procedure_concept_id=row['surgery_type']
    procedure_concept_id=cols[procedure_concept_id]
    procedure_date=row['date_of_surgery']
    procedure_type_concept_id=32879 #Still dont know what to do with this one
    curIDEA.execute(queryDate, (row['episode_event']))
    res=curIDEA.fetchone()
    dateAux=res
    cur.execute(sqlGetEpisode, (person_id,dateAux))
    epId=cur.fetchone()
    newId = datetime.now().strftime('%Y%m%d%H%M%S') + str(uuid4())
    toTableEpisodeEventProcedureOcurrence(epId,newId)
    provider_id=row['surgery_hospital']
    cur.execute(sqlProcedure,(newId, person_id, procedure_concept_id,procedure_date,procedure_type_concept_id))
    conn.commit()
    row['newId']=newId


### Surgery to Measurement

Measurement concept id?

Should these be "modifiers" of the main surgery, of the other surgeries, or none???

In [ ]:

cols= {
    "SURGICAL_PROCEDURE": 4301351,
    "LOCAL_EXCISION": 4315400,
    "AMPUTATION": 4217482,
    "LIMB_OPERATION": 4181193,
    "PALLIATIVE": 4179711,
    "CURATIVE": 4162591,
    "EXTERNAL_OPEN": 4236080,
    "TRANSNASAL": 4050374,
    "TRANSORBITAL": 4044897,
    "TRANSORAL": 4038679,
    "CRANIOFACIAL": 4333339,
    "NO_RESIDUAL_TUMOR": 1634643,
    "MICROSCOPIC_RESIDUAL_TUMOR": 1633801,
    "MACROSCOPIC_RESIDUAL_TUMOR": 1634484,
    "tumor_rupture": 36768904, #Since we have a vocabulary for a boolean, I will use it as a modifier
    "surgical_specimen_mitotic_count": 4227243,
    "GRADE_1": 1634371,
    "GRADE_2": 1634752,
    "GRADE_3": 1633749,
    "GRADE_H": 1635587,
    "GRADE_L": 1634085,
    "reconstruction": 3010988,
    "IPSILATERAL": 4112106,
    "BILATERAL_SAMPLING": 4117496,
    "CONTROLATERAL": 4112107,
    "site_of_surgery_on_metastasis_soft_tissue": 35225724,
    "site_of_surgery_on_metastasis_distant_lymph_nodes": 36769243,
    "site_of_surgery_on_metastasis_lung": 36770283,
    "site_of_surgery_on_metastasis_bone": 36769301,
    "site_of_surgery_on_metastasis_liver": 36770544,
    "site_of_surgery_on_metastasis_pleura": 35226258,             
    "site_of_surgery_on_metastasis_peritoneum": 35226253,
    "site_of_surgery_on_metastasis_brain": 36768862,
    "site_of_surgery_on_metastasis_other_viscera":  36769180,
    "site_of_surgery_on_metastasis_unknown": 4129922,
    "NO_COMPLICATION": 4132135,
    "CLA_DINDO_SCALE_I": 715974,
    "CLA_DINDO_SCALE_II": 715975,
    "CLA_DINDO_SCALE_III": 715976,
    "CLA_DINDO_SCALE_IIIA": 715977,
    "CLA_DINDO_SCALE_IIIB": 715978,
    "CLA_DINDO_SCALE_IV": 715979,
    "CLA_DINDO_SCALE_IVA": 715980,
    "CLA_DINDO_SCALE_IVB": 715981,
    "CLA_DINDO_SCALE_V": 715982
}

col_Names=[
    "isolated_limb_perfusion",
    "laterality_of_the_dissection",
    "margins_after_surgery",
    "neck_surgery",
    "reconstruction",
    "surgery_intention",
    "surgery_onm",
    "surgical_complications",
    "surgical_specimen_grading_only_in_untreated_tumours",
    "surgical_specimen_mitotic_count",
    "tumor_rupture",
    "type_of_surgical_approach_on_tumour"
]
metastasis_cols=[
    "site_of_surgery_on_metastasis_bone",
    "site_of_surgery_on_metastasis_brain",
    "site_of_surgery_on_metastasis_distant_lymph_nodes",
    "site_of_surgery_on_metastasis_liver",
    "site_of_surgery_on_metastasis_lung",
    "site_of_surgery_on_metastasis_other_viscera",
    "site_of_surgery_on_metastasis_peritoneum",
    "site_of_surgery_on_metastasis_pleura",
    "site_of_surgery_on_metastasis_soft_tissue",
    "site_of_surgery_on_metastasis_unknown"]

sql =  """
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_number,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s);
    """
query= """
    SELECT c.patient 
    FROM cancer_episode c
    WHERE EXISTS (
        SELECT 1 
        FROM EpisodeEvent e
        WHERE e.cancerEpisode = c.id
        AND e.id = %s
    )
    LIMIT 1;

"""
insertMeasValue="""
    INSERT INTO omopcdm.measurement (person_id, measurement_concept_id, measurement_date, measurement_type_concept_id, value_as_concept_id,measurement_event_id,meas_event_field_concept_id)
    VALUES (%s, %s, %s,%s,%s,%s,%s);
"""
meas_event_field_concept_id=1147082

boolValues=[4188539,4188540]
measurement_type_concept_id=32879
for idx, row in df_tables.iterrows():
    for column in col_Names:
        if column == "neck_surgery":
            if row[column] or row[column]=="YES":
                value_as_concept_id=45884591 #Yes, surgery
                concept_id=36770046
                curIDEA.execute(query, (row['episode_event']))
                person_id=curIDEA.fetchone()
                cur.execute(insertMeasValue,(person_id,concept_id,row['date_of_neck_surgey'],measurement_type_concept_id,value_as_concept_id,row['newId'],meas_event_field_concept_id))
                conn.commit()
        elif column== "surgery_onm":
            if row[column] or row[column]=="YES":
                value_as_concept_id=45884591 #Yes, surgery
                for metastasi in metastasis_cols:
                    if row[metastasi]:
                        concept_id=cols[metastasi]
                        curIDEA.execute(query, (row['episode_event']))
                        person_id=curIDEA.fetchone()
                        cur.execute(insertMeasValue,(person_id,concept_id,row['date_of_surgery_onm'],measurement_type_concept_id,value_as_concept_id,row['newId'],meas_event_field_concept_id))
                        conn.commit()
        else:
            if column in ("reconstruction", "isolated_limb_perfusion", ):
                if row[column] or row[column]=="YES":
                    value_as_concept_id=4188539
                    concept_id=cols[column]
                else:
                    value_as_concept_id=4188540
                    concept_id=cols[column]

                curIDEA.execute(query, (row['episode_event']))
                person_id=curIDEA.fetchone()
                cur.execute(insertMeasValue,(person_id,concept_id,row['date_of_surgery'],measurement_type_concept_id,value_as_concept_id,row['newId'],meas_event_field_concept_id))
                conn.commit()
            else:
                concept_id=cols[column]
                value=row[column]
                curIDEA.execute(query, (row['episode_event']))
                person_id=curIDEA.fetchone()
                cur.execute(sql,(person_id,concept_id,row['date_of_surgery'],measurement_type_concept_id,value,row['newId'],meas_event_field_concept_id))
                conn.commit()


                

